In [ ]:
import mysql.connector
from matplotlib import pyplot as plt

# returns connection object #
def connect_to_db(db_name):
    cnx = mysql.connector.connect(
    user='root',
    password='chalkHorseMountain',
    host='localhost',
    database=db_name
    )
    return cnx

# returns boolean #
def table_exists(cursor, tbl_name):
    cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_schema = DATABASE()
        AND table_name = \"""" + tbl_name + """\";
    """)
    
    if cursor.fetchone()[0] == 1:
        return True
    return False

class stock_hist(object):
    def __init__(self, symbol, votes, comments, dates):
        self.symbol = symbol
        self.votes = votes
        self.comments = comments
        self.dates = dates

def get_hist(symbol, date_range):
    
    cnx = connect_to_db("TheSpatula")
    mycursor = cnx.cursor()
    assert mycursor
    assert table_exists(mycursor, "reddit")
    assert table_exists(mycursor, "youtube")
    
    mycursor.execute(f"""
        SELECT num_votes, num_comments, date_posted
        FROM reddit
        WHERE symbol="{symbol}"
        AND date_posted > "{date_range[0]}"
        AND date_posted < "{date_range[1]}"
        ;
        """)
    
    votes = list()
    comments = list()
    dates = list()
    
    for (num_votes, num_comments, date_posted) in mycursor:
        votes.append(num_votes)
        comments.append(num_comments)
        dates.append(date_posted)
    
    mycursor.close()
    cnx.close()
    
    return stock_hist(symbol, votes, comments, dates)

def plot_hist(symbol, date_range):
    hist = get_hist(symbol, date_range)
    
    plt.plot(hist.dates, hist.votes)
    plt.plot(hist.dates, hist.comments)
    plt.title(f"{symbol} Chart")
    plt.xlabel("Date Posted")
    plt.ylabel("Votes and Comments")
    plt.legend(["Votes", "Comments"])
    
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)

if __name__ == "__main__":
    
    plot_hist("AMC", ("2021-05-20", "2021-06-02"))
    